<a href="https://colab.research.google.com/github/fantaxiah/Dissertation/blob/main/AC_image_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
''

# README:
* Roboflow and Real-ESRGAN need two different versions of torch and torchvision.

* And the denoising functions needs a different version of opencv than what is on Colab.

* You will need to restart the session twice, once at the beginning and once in the middle.

* Use your own roboflow API key.

* AC-locks--2 should save to your own environment.

* The images and enhanced_images folders can be interchanged in the real-ESRGAN model.



In [ ]:
!pip install opencv-contrib-python==4.10.0.82
!pip install numpy==1.24.4 # Explicitly install a compatible numpy version

### *Restart*

# Install Roboflow

In [ ]:
!pip install -q ultralytics roboflow supervision

In [ ]:
# !pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision #Install roboflow supervision

In [ ]:
import os

from google.colab import userdata

os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY") #This API key comes from roboflow website


# Download Dataset

In [ ]:
from roboflow import download_dataset

dataset = download_dataset('https://universe.roboflow.com/ac-locks/ac-locks/dataset/2', 'yolov8')

In [ ]:
print("AC-locks--2 folder contents:", os.listdir('AC-locks--2'))

In [ ]:
print(os.listdir('AC-locks--2/train'))

# Denoising and Sharpening

In [ ]:
import cv2
import numpy as np


for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error reading {img_path}")
            continue

        # Denoising
        denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

        # Sharpening
        sharpening_kernel = np.array([[0, -1, 0],
                                      [-1, 5, -1],
                                      [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpening_kernel)

        # Histogram Equalization
        img_yuv = cv2.cvtColor(sharpened, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        enhanced_img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, enhanced_img)


In [ ]:
import matplotlib.pyplot as plt

def compare_images(original_dir, enhanced_dir, num_images=5):
    original_images = sorted([f for f in os.listdir(original_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
    enhanced_images = sorted([f for f in os.listdir(enhanced_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])

    # Show that images that exist in both folders
    common_files = list(set(original_images) & set(enhanced_images))

    if len(common_files) == 0:
        print("No matching images found in both directories.")
        return

    for i, filename in enumerate(common_files[:num_images]):
        original_path = os.path.join(original_dir, filename)
        enhanced_path = os.path.join(enhanced_dir, filename)

        original_img = cv2.imread(original_path)
        enhanced_img = cv2.imread(enhanced_path)

        if original_img is None or enhanced_img is None:
            print(f"Skipping {filename} - could not read image.")
            continue

        # Convert BGR to RGB
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        enhanced_img = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.title(f'Original: {filename}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(enhanced_img)
        plt.title(f'Enhanced: {filename}')
        plt.axis('off')

        plt.show()

In [ ]:
original_dir = 'AC-locks--2/train/images'
enhanced_dir = 'AC-locks--2/train/enhanced_images'

compare_images(original_dir, enhanced_dir, num_images=5)

# Super resolution

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 -f https://download.pytorch.org/whl/torch_stable.html

### *Restart session*

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
import os
from google.colab import files
import shutil
import cv2
import matplotlib.pyplot as plt

# Define the directory
image_folder = '/content/AC-locks--2/train/images'
result_folder = '/content/AC-locks--2/train/high_res_images'

# Create the result folder if it doesn't exist
os.makedirs(result_folder, exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Loop through each image
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f'Image: {img_file}')
    plt.axis('off')

'''
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)
'''

In [ ]:
!python /content/Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i /content/AC-locks--2/train/images -o /content/AC-locks--2/train/high_res_images --tile 128 --fp32 --suffix ""

In [ ]:
# utils for visualization
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('Real-ESRGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import glob

input_folder = '/content/AC-locks--2/train/images'
result_folder = '/content/AC-locks--2/train/high_res_images'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)